`pidgin.code_formatting` controls pygments specific formatting opinions for `pidgin`.

In [1]:
    import pygments.formatters.other, IPython, html, nbconvert, io, pathlib, re, os

`format_title` replaces newlines with valid in inline html delimiters.

https://stackoverflow.com/questions/18606877/how-can-i-new-newline-in-title-attribute-in-html

In [2]:
    def format_title(object): 
        return '&#013;'.join(html.escape(nbconvert.filters.strip_ansi(str(object))).splitlines())

In [3]:
    def sanitize(s: str)->str:
        """>>> assert sanitize(ip.object_inspect_text('IPython'))"""
        buffer = io.StringIO()
        for i, segment in enumerate(re.split("([\'\"\s]+)", s)):
            if (os.sep in segment) and ('.' in segment):
                path = pathlib.Path(segment)
                if path.suffix: segment = F"'<safe path>{path.name}'"
            buffer.write(segment)
        try: return buffer.getvalue()
        finally: buffer.close()

`EscapedFormatter` allows html literals to be injected into `pygements`.

In [4]:
    class EscapedFormatter:
        def format(self, tokensource, outfile):
            _escape_html_table = pygments.formatters.html._escape_html_table 
            pygments.formatters.html._escape_html_table = {}
            try: return super().format(tokensource, outfile)
            finally: pygments.formatters.html._escape_html_table = _escape_html_table

`NamingLexer` combines dotted paths in the code to represent as a single object.

In [5]:
    class NamingLexer(pygments.lexers.PythonLexer):
        def get_tokens(self, text, unfiltered=False):
            buffer_token, buffer = None, "''"
            for token, snippet in list(super().get_tokens(text)) + [
                (pygments.token.Token.Generic, '')
            ]:
                if buffer and ((snippet == '.') or buffer.endswith('.')):  
                    buffer += snippet
                elif token in {
                    pygments.token.Token.Name, pygments.token.Token.Name.Namespace, 
                    pygments.token.Token.Name.Class, pygments.token.Token.Name.Function
                }:
                    buffer_token, buffer = token, snippet            
                else:
                    if buffer_token and buffer: 
                        try: 
                            title = IPython.get_ipython().object_inspect_text(
                                buffer, detail_level=buffer_token in {pygments.token.Token.Name.Function}
                            ) 
                            try:
                                if ("PYTEST_CURRENT_TEST" in __import__('os').environ) or IPython.get_ipython().weave.safe:
                                    title = sanitize(title)
                            except AttributeError: assert True, """Suboptimal"""
                            
                            yield buffer_token, F"""<span style="{title and "border-bottom: 1px dotted gray;" or ""
                            }" title="{format_title(title)}">{buffer}</span>"""
                        except: 
                            yield buffer_token, buffer
                        buffer_token, buffer = None, ""
                    yield token, snippet

In [6]:
    def markup_code(source, format=pygments.formatters.other.NullFormatter):
        """
        <style>{{pygments.formatters.HtmlFormatter().get_style_defs()}}</style>        
        """
        return pygments.highlight(source, NamingLexer(), type('formatter', (EscapedFormatter, format), {})())

In [7]:
    def markup_code_inline(source):
        """Broken atm cause of formatting conflicts with rise.
        <style>{{pygments.formatters.HtmlFormatter().get_style_defs()}}</style>        
        """
        inline
        return (markup_code(source))

In [8]:
    def inline(html):
        lead, sep, html = html.partition('div')
        html = lead + 'span' + html
        lead, sep, html = html.rpartition('/div')
        html = lead + '/span' + html
        lead, sep, html = html.partition('<pre>')
        html = lead + html
        lead, sep, html = html.partition('</pre>')
        return lead + html